In [1]:
import pandas as pd
import sqlalchemy as sa
import psycopg2 as pg
from config import *

In [2]:
sa_conn_str = f'postgresql+psycopg2://{user}:{password}@{host}/{database}'
engine = sa.create_engine(sa_conn_str)
conn = engine.connect()

In [3]:
conn_str = f"host={host} dbname={database} user={user} password={password}"
try:
    conn_pg = pg.connect(conn_str)
    print("Connection established")
except:
    print("Connection failed")

Connection established


In [4]:
with pg.connect(conn_str) as conn_pg:
    c = conn_pg.cursor()
    pg_create_table = '''
    CREATE TABLE IF NOT EXISTS emp_og (
        EEID char(6),
        FULL_NAME varchar(50),
        Job_Title varchar(50),
        Department varchar(50),
        Business_Unit varchar(50),
        Gender varchar(50),
        Ethnicity varchar(50),
        Age int,
        Hire_Date date,
        Annual_Salary int,
        Bonus_percent int,
        Country varchar(50),
        City varchar(50)
    );'''
    c.execute(pg_create_table)
    conn_pg.commit()

with pg.connect(conn_str) as conn_pg:
    c = conn_pg.cursor()
    pg_create_table = '''
    CREATE TABLE IF NOT EXISTS emp_err (
        EEID char(6),
        FULL_NAME varchar(50),
        Job_Title varchar(50),
        Department varchar(50),
        Business_Unit varchar(50),
        Gender varchar(50),
        Ethnicity varchar(50),
        Age int,
        Hire_Date date,
        Annual_Salary int,
        Bonus_percent int,
        Country varchar(50),
        City varchar(50)
    );'''
    c.execute(pg_create_table)
    conn_pg.commit()

In [5]:
emp_og = pd.read_sql(sa.text('SELECT * FROM emp_og'), conn)
emp_og.head()

,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city


In [6]:
emp_err = pd.read_sql(sa.text('SELECT * FROM emp_err'), conn)
emp_err.head()

,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city


In [7]:
# READING OUR DATA FROM CSV FILE

data_OG = pd.read_csv('employee_data_OG.csv')
data_OG.columns = emp_og.columns

data_OG.age = data_OG.age.astype(int)
data_OG.annual_salary = data_OG.annual_salary.str.replace('$', '').str.replace(',', '').astype(int)
data_OG.bonus_percent = data_OG.bonus_percent.str.replace('%', '').astype(int)
data_OG.hire_date = pd.to_datetime(data_OG.hire_date)

data_OG.head()

C:\Users\psg44\AppData\Local\Temp\ipykernel_14432\2100916667.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_OG.annual_salary = data_OG.annual_salary.str.replace('$', '').str.replace(',', '').astype(int)


,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604,15,United States,Seattle
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975,0,China,Chongqing
2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099,20,United States,Chicago
3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913,7,United States,Chicago
4,E01639,Austin Vo,Sr. Analyst,Finance,Manufacturing,Male,Asian,55,1995-11-20,95409,0,United States,Phoenix


In [8]:
# data_OG.to_sql('emp_og', con = engine, if_exists='replace', index=False)

In [9]:
data_err = pd.read_csv('employee_data_ERRORed.csv')
data_err.columns = data_OG.columns

data_err.age = data_err.age.astype(int)
data_err.annual_salary = data_err.annual_salary.str.replace('$', '').str.replace(',', '').astype(int)
data_err.bonus_percent = data_err.bonus_percent.str.replace('%', '').astype(int)
data_err.hire_date = pd.to_datetime(data_err.hire_date)

data_err.head()

C:\Users\psg44\AppData\Local\Temp\ipykernel_14432\960684199.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_err.annual_salary = data_err.annual_salary.str.replace('$', '').str.replace(',', '').astype(int)


,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604,15,United States,Seattle
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975,0,China,Chongqing
2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099,20,United States,Chicago
3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913,7,United States,Chicago
4,E01639,Austin Vo,Sr. Analyst,Finance,Manufacturing,Male,Asian,55,1995-11-20,95409,0,United States,Phoenix


In [10]:
# data_err.to_sql('emp_og', con = engine, if_exists='replace', index=False)

## Reading from the database

Looks like ```pd.DataFrame.to_sql()``` didn't work at all... Let's try psycopg2

In [11]:
import psycopg2
import psycopg2.extras as extras

def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [12]:
with pg.connect(conn_str) as conn_pg:
    execute_values(conn_pg, data_OG, 'emp_og')

with pg.connect(conn_str) as conn_pg:
    execute_values(conn_pg, data_err, 'emp_err')

the dataframe is inserted
the dataframe is inserted


In [13]:
emp_og = pd.read_sql(sa.text('SELECT * FROM emp_og'), conn)
emp_og.head()

,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604,15,United States,Seattle
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975,0,China,Chongqing
2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099,20,United States,Chicago
3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913,7,United States,Chicago
4,E01639,Austin Vo,Sr. Analyst,Finance,Manufacturing,Male,Asian,55,1995-11-20,95409,0,United States,Phoenix


In [14]:
emp_err = pd.read_sql(sa.text('SELECT * FROM emp_err'), conn)
emp_err.head()

,eeid,full_name,job_title,department,business_unit,gender,ethnicity,age,hire_date,annual_salary,bonus_percent,country,city
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,Black,55,2016-04-08,141604,15,United States,Seattle
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,Asian,59,1997-11-29,99975,0,China,Chongqing
2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,Caucasian,50,2006-10-26,163099,20,United States,Chicago
3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,Caucasian,26,2019-09-27,84913,7,United States,Chicago
4,E01639,Austin Vo,Sr. Analyst,Finance,Manufacturing,Male,Asian,55,1995-11-20,95409,0,United States,Phoenix


Hence, we've inserted some CSV data into the database. Now let's try and compare the original and the error data.

In [15]:
emp_og.compare(emp_err)

full_name           department                   business_unit  \
            self    other        self      other                 self   
7    Luke Martin  Shivraj         NaN        NaN                  NaN   
9            NaN      NaN     Finance         IT                  NaN   
11           NaN      NaN         NaN        NaN  Speciality Products   
999          NaN      NaN  Accounting  Marketing                  NaN   

                    
             other  
7              NaN  
9              NaN  
11   Manufacturing  
999            NaN